In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from torchmetrics.regression import MeanSquaredError

import optree

from astra.torch.models import MLPRegressor
from astra.torch.utils import train_fn
from astra.torch.data import load_cifar_10, load_mnist

/home/patel_zeel/ASTRA/astra/torch/data.py:12: UserWarning: TORCH_HOME not set, setting it to /home/patel_zeel/.cache/torch
  warnings.warn(f"TORCH_HOME not set, setting it to {os.environ['TORCH_HOME']}")


In [4]:
ds = load_mnist()
ds


MNIST Dataset
length of dataset: 70000
shape of images: torch.Size([28, 28])
len of classes: 10
classes: ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
dtype of images: torch.float32
dtype of labels: torch.int64
range of image value: min=0.0, max=1.0

In [3]:
model = MLPRegressor(2, [5, 10], 1)
x = torch.rand(10, 2)
y = torch.rand(10, 1)
# opt = torch.optim.Adam(model.parameters(), lr=0.01)

iter_losses, epoch_losses = train_fn(
    model, input=x, output=y, loss_fn=nn.MSELoss(), epochs=5, batch_size=2, shuffle=True, 
)

# for _ in range(5):
#     opt.zero_grad()
#     out = model(x)
#     loss = F.mse_loss(out, y)
#     print(loss)
#     loss.backward()
#     opt.step()

100%|██████████| 5/5 [00:00<00:00, 20.96it/s]


Epoch 1: 0.3687124252319336


100%|██████████| 5/5 [00:00<00:00, 920.17it/s]


Epoch 2: 0.2589665248990059


100%|██████████| 5/5 [00:00<00:00, 756.93it/s]


Epoch 3: 0.18627891764044763


100%|██████████| 5/5 [00:00<00:00, 1158.52it/s]


Epoch 4: 0.12760575022548437


100%|██████████| 5/5 [00:00<00:00, 1254.73it/s]

Epoch 5: 0.0931696355342865
